# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 
Based on the paper titled "Show and Tell: A Neural Image Caption Generator" i choose to have an Encoder - Decoder Architecture, we start by Encoder which is a **pre-trained** CNN with a classification task, then we remove the last layer and we add a fully connected layer, wish is the features encoder, then we concatinate it to an Decoder, which a **non-trained** LSTM which result each time a one-hot vector with more than 8000 value, the value are mapped to a dictionary of words predefined already. than we apply a arg-sigmoid to get the predicted world each time. 

### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 
I kept the parameters as they are:
- The task is to describe a photo so i don't think we can downsample the image more than 256x256, if we do more we will lose details
- i keept the image treatments as : RandomCrop, RandomHorizontalFlip, Normalize as they will be used as data augmentation and avoiding overfiting.
- as batch size i putted 32, it is within the recommended range.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 
i started by basic values as 256 for embeded size, 512 for hidden size and 32 as batch size and depends on the result of the model i will decide to keep or change them

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** i used Adam optimiser as it fix well the overfiting problem.

In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 32          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True     # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 10             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

# TODO #4: Define the optimizer.
# criterion = nn.MSELoss()

optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.89s)
creating index...
index created!
Obtaining caption lengths...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 329/414113 [00:00<02:05, 3284.58it/s]

  0%|          | 776/414113 [00:00<01:55, 3567.24it/s]

  0%|          | 1235/414113 [00:00<01:48, 3820.72it/s]

  0%|          | 1689/414113 [00:00<01:42, 4010.14it/s]

  1%|          | 2142/414113 [00:00<01:39, 4151.44it/s]

  1%|          | 2594/414113 [00:00<01:36, 4253.17it/s]

  1%|          | 3041/414113 [00:00<01:35, 4313.87it/s]

  1%|          | 3481/414113 [00:00<01:34, 4338.79it/s]

  1%|          | 3939/414113 [00:00<01:33, 4406.09it/s]

  1%|          | 4399/414113 [00:01<01:31, 4460.20it/s]

  1%|          | 4853/414113 [00:01<01:31, 4483.14it/s]

  1%|▏         | 5317/414113 [00:01<01:30, 4527.59it/s]

  1%|▏         | 5775/414113 [00:01<01:29, 4540.45it/s]

  2%|▏         | 6231/414113 [00:01<01:29, 4543.62it/s]

  2%|▏         | 6695/414113 [00:01<01:29, 4569.93it/s]

  2%|▏         | 7151/414113 [00:01<01:29, 4542.98it/s]

  2%|▏         | 7608/414113 [00:01<01:29, 4

 14%|█▍        | 58830/414113 [00:13<02:17, 2582.56it/s]

 14%|█▍        | 59291/414113 [00:13<01:59, 2973.97it/s]

 14%|█▍        | 59750/414113 [00:13<01:46, 3324.77it/s]

 15%|█▍        | 60211/414113 [00:13<01:37, 3627.21it/s]

 15%|█▍        | 60673/414113 [00:13<01:31, 3875.73it/s]

 15%|█▍        | 61110/414113 [00:13<01:28, 4010.44it/s]

 15%|█▍        | 61568/414113 [00:13<01:24, 4164.23it/s]

 15%|█▍        | 62028/414113 [00:13<01:22, 4284.66it/s]

 15%|█▌        | 62481/414113 [00:13<01:20, 4352.33it/s]

 15%|█▌        | 62944/414113 [00:14<01:19, 4430.00it/s]

 15%|█▌        | 63404/414113 [00:14<01:18, 4477.83it/s]

 15%|█▌        | 63866/414113 [00:14<01:17, 4517.81it/s]

 16%|█▌        | 64323/414113 [00:14<01:17, 4530.80it/s]

 16%|█▌        | 64780/414113 [00:14<01:17, 4527.85it/s]

 16%|█▌        | 65241/414113 [00:14<01:16, 4550.02it/s]

 16%|█▌        | 65700/414113 [00:14<01:16, 4561.02it/s]

 16%|█▌        | 66164/414113 [00:14<01:15, 4582.99it/s]

 16%|█▌       

 28%|██▊       | 117627/414113 [00:25<01:04, 4574.96it/s]

 29%|██▊       | 118098/414113 [00:26<01:04, 4614.61it/s]

 29%|██▊       | 118560/414113 [00:26<01:04, 4607.96it/s]

 29%|██▊       | 119026/414113 [00:26<01:03, 4623.24it/s]

 29%|██▉       | 119496/414113 [00:26<01:03, 4644.54it/s]

 29%|██▉       | 119961/414113 [00:26<01:03, 4617.87it/s]

 29%|██▉       | 120423/414113 [00:26<01:04, 4545.62it/s]

 29%|██▉       | 120878/414113 [00:26<01:04, 4542.01it/s]

 29%|██▉       | 121344/414113 [00:26<01:03, 4575.57it/s]

 29%|██▉       | 121802/414113 [00:26<01:03, 4575.79it/s]

 30%|██▉       | 122266/414113 [00:26<01:03, 4594.54it/s]

 30%|██▉       | 122726/414113 [00:27<01:03, 4572.28it/s]

 30%|██▉       | 123184/414113 [00:27<01:03, 4547.90it/s]

 30%|██▉       | 123641/414113 [00:27<01:03, 4553.24it/s]

 30%|██▉       | 124116/414113 [00:27<01:02, 4609.50it/s]

 30%|███       | 124588/414113 [00:27<01:02, 4640.33it/s]

 30%|███       | 125066/414113 [00:27<01:01, 4680.60it/s

 43%|████▎     | 176297/414113 [00:39<01:32, 2559.03it/s]

 43%|████▎     | 176765/414113 [00:39<01:20, 2961.34it/s]

 43%|████▎     | 177234/414113 [00:39<01:11, 3328.39it/s]

 43%|████▎     | 177702/414113 [00:39<01:04, 3643.25it/s]

 43%|████▎     | 178162/414113 [00:39<01:00, 3883.96it/s]

 43%|████▎     | 178625/414113 [00:39<00:57, 4079.47it/s]

 43%|████▎     | 179090/414113 [00:39<00:55, 4234.79it/s]

 43%|████▎     | 179556/414113 [00:39<00:53, 4351.09it/s]

 43%|████▎     | 180016/414113 [00:39<00:52, 4422.00it/s]

 44%|████▎     | 180474/414113 [00:39<00:52, 4454.76it/s]

 44%|████▎     | 180931/414113 [00:40<00:51, 4487.74it/s]

 44%|████▍     | 181397/414113 [00:40<00:51, 4537.00it/s]

 44%|████▍     | 181857/414113 [00:40<00:51, 4477.37it/s]

 44%|████▍     | 182325/414113 [00:40<00:51, 4534.35it/s]

 44%|████▍     | 182782/414113 [00:40<00:50, 4541.49it/s]

 44%|████▍     | 183246/414113 [00:40<00:50, 4567.82it/s]

 44%|████▍     | 183705/414113 [00:40<00:50, 4561.62it/s

 57%|█████▋    | 234565/414113 [00:51<00:39, 4540.84it/s]

 57%|█████▋    | 235020/414113 [00:51<00:39, 4543.45it/s]

 57%|█████▋    | 235475/414113 [00:51<00:39, 4510.86it/s]

 57%|█████▋    | 235927/414113 [00:52<00:39, 4496.32it/s]

 57%|█████▋    | 236380/414113 [00:52<00:39, 4504.18it/s]

 57%|█████▋    | 236833/414113 [00:52<00:39, 4511.33it/s]

 57%|█████▋    | 237295/414113 [00:52<00:38, 4543.16it/s]

 57%|█████▋    | 237753/414113 [00:52<00:38, 4552.11it/s]

 58%|█████▊    | 238213/414113 [00:52<00:38, 4566.34it/s]

 58%|█████▊    | 238670/414113 [00:52<00:38, 4542.74it/s]

 58%|█████▊    | 239125/414113 [00:52<00:38, 4531.29it/s]

 58%|█████▊    | 239582/414113 [00:52<00:38, 4541.16it/s]

 58%|█████▊    | 240039/414113 [00:52<00:38, 4548.70it/s]

 58%|█████▊    | 240496/414113 [00:53<00:38, 4552.89it/s]

 58%|█████▊    | 240961/414113 [00:53<00:37, 4579.67it/s]

 58%|█████▊    | 241421/414113 [00:53<00:37, 4585.02it/s]

 58%|█████▊    | 241880/414113 [00:53<00:37, 4534.46it/s

 70%|███████   | 291835/414113 [01:04<00:26, 4552.41it/s]

 71%|███████   | 292293/414113 [01:04<00:26, 4559.08it/s]

 71%|███████   | 292749/414113 [01:04<00:26, 4551.39it/s]

 71%|███████   | 293219/414113 [01:04<00:26, 4593.10it/s]

 71%|███████   | 293679/414113 [01:04<00:26, 4574.05it/s]

 71%|███████   | 294137/414113 [01:04<00:26, 4574.51it/s]

 71%|███████   | 294600/414113 [01:05<00:26, 4589.62it/s]

 71%|███████▏  | 295060/414113 [01:05<00:25, 4589.58it/s]

 71%|███████▏  | 295529/414113 [01:05<00:25, 4617.74it/s]

 71%|███████▏  | 295991/414113 [01:05<00:25, 4595.23it/s]

 72%|███████▏  | 296451/414113 [01:05<00:25, 4562.23it/s]

 72%|███████▏  | 296908/414113 [01:05<00:25, 4563.63it/s]

 72%|███████▏  | 297365/414113 [01:05<00:26, 4465.42it/s]

 72%|███████▏  | 297813/414113 [01:05<00:26, 4413.89it/s]

 72%|███████▏  | 298260/414113 [01:05<00:26, 4430.13it/s]

 72%|███████▏  | 298725/414113 [01:06<00:25, 4493.58it/s]

 72%|███████▏  | 299189/414113 [01:06<00:25, 4534.79it/s

 84%|████████▍ | 349318/414113 [01:17<00:14, 4529.86it/s]

 84%|████████▍ | 349773/414113 [01:17<00:14, 4535.19it/s]

 85%|████████▍ | 350227/414113 [01:17<00:14, 4503.99it/s]

 85%|████████▍ | 350690/414113 [01:17<00:13, 4538.65it/s]

 85%|████████▍ | 351161/414113 [01:18<00:13, 4586.57it/s]

 85%|████████▍ | 351620/414113 [01:18<00:13, 4569.00it/s]

 85%|████████▌ | 352078/414113 [01:18<00:13, 4562.30it/s]

 85%|████████▌ | 352543/414113 [01:18<00:13, 4587.77it/s]

 85%|████████▌ | 353002/414113 [01:18<00:13, 4568.89it/s]

 85%|████████▌ | 353459/414113 [01:18<00:13, 4559.11it/s]

 85%|████████▌ | 353922/414113 [01:18<00:13, 4578.49it/s]

 86%|████████▌ | 354386/414113 [01:18<00:12, 4595.13it/s]

 86%|████████▌ | 354853/414113 [01:18<00:12, 4617.10it/s]

 86%|████████▌ | 355315/414113 [01:18<00:12, 4617.88it/s]

 86%|████████▌ | 355778/414113 [01:19<00:12, 4618.75it/s]

 86%|████████▌ | 356240/414113 [01:19<00:12, 4587.97it/s]

 86%|████████▌ | 356703/414113 [01:19<00:12, 4597.76it/s

 98%|█████████▊| 407175/414113 [01:30<00:01, 4515.49it/s]

 98%|█████████▊| 407635/414113 [01:30<00:01, 4540.24it/s]

 99%|█████████▊| 408090/414113 [01:30<00:01, 4530.47it/s]

 99%|█████████▊| 408544/414113 [01:30<00:01, 4483.72it/s]

 99%|█████████▉| 408993/414113 [01:30<00:01, 4478.37it/s]

 99%|█████████▉| 409456/414113 [01:30<00:01, 4521.74it/s]

 99%|█████████▉| 409914/414113 [01:30<00:00, 4537.81it/s]

 99%|█████████▉| 410377/414113 [01:31<00:00, 4564.17it/s]

 99%|█████████▉| 410842/414113 [01:31<00:00, 4588.36it/s]

 99%|█████████▉| 411301/414113 [01:31<00:00, 4577.54it/s]

 99%|█████████▉| 411759/414113 [01:31<00:00, 4529.83it/s]

100%|█████████▉| 412213/414113 [01:31<00:00, 4506.88it/s]

100%|█████████▉| 412664/414113 [01:31<00:00, 4493.94it/s]

100%|█████████▉| 413114/414113 [01:31<00:00, 4489.67it/s]

100%|█████████▉| 413579/414113 [01:31<00:00, 4533.89it/s]

100%|█████████▉| 414039/414113 [01:31<00:00, 4551.91it/s]

100%|██████████| 414113/414113 [01:31<00:00, 4504.86it/s

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/10], Step [100/12942], Loss: 4.4091, Perplexity: 82.1931
Epoch [1/10], Step [200/12942], Loss: 3.7611, Perplexity: 42.9942
Epoch [1/10], Step [300/12942], Loss: 3.7205, Perplexity: 41.28469
Epoch [1/10], Step [400/12942], Loss: 3.3173, Perplexity: 27.5852
Epoch [1/10], Step [500/12942], Loss: 3.1490, Perplexity: 23.3127
Epoch [1/10], Step [600/12942], Loss: 3.0183, Perplexity: 20.4559
Epoch [1/10], Step [700/12942], Loss: 3.5464, Perplexity: 34.6896
Epoch [1/10], Step [800/12942], Loss: 2.9442, Perplexity: 18.9955
Epoch [1/10], Step [900/12942], Loss: 3.2777, Perplexity: 26.5145
Epoch [1/10], Step [1000/12942], Loss: 2.9845, Perplexity: 19.7764
Epoch [1/10], Step [1100/12942], Loss: 2.8793, Perplexity: 17.8014
Epoch [1/10], Step [1200/12942], Loss: 2.9424, Perplexity: 18.9604
Epoch [1/10], Step [1300/12942], Loss: 2.8813, Perplexity: 17.8376
Epoch [1/10], Step [1400/12942], Loss: 2.7193, Perplexity: 15.1692
Epoch [1/10], Step [1600/12942], Loss: 2.4754, Perplexity: 11.8869
Epo

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch [1/10], Step [6600/12942], Loss: 2.2700, Perplexity: 9.67942
Epoch [1/10], Step [6700/12942], Loss: 2.3717, Perplexity: 10.7152
Epoch [1/10], Step [6800/12942], Loss: 2.1501, Perplexity: 8.58552
Epoch [1/10], Step [6900/12942], Loss: 2.1324, Perplexity: 8.43520
Epoch [1/10], Step [7000/12942], Loss: 2.0584, Perplexity: 7.83330
Epoch [1/10], Step [7100/12942], Loss: 2.4558, Perplexity: 11.6553
Epoch [1/10], Step [7200/12942], Loss: 2.1563, Perplexity: 8.63957
Epoch [1/10], Step [7300/12942], Loss: 2.2473, Perplexity: 9.46236
Epoch [1/10], Step [7400/12942], Loss: 2.4605, Perplexity: 11.7102
Epoch [1/10], Step [7500/12942], Loss: 2.0349, Perplexity: 7.65131
Epoch [1/10], Step [7600/12942], Loss: 2.0097, Perplexity: 7.46089
Epoch [1/10], Step [7700/12942], Loss: 2.9015, Perplexity: 18.2022
Epoch [1/10], Step [7800/12942], Loss: 1.9876, Perplexity: 7.29797
Epoch [1/10], Step [7900/12942], Loss: 2.1604, Perplexity: 8.67426
Epoch [1/10], Step [8000/12942], Loss: 2.2094, Perplexity: 9.1

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch [1/10], Step [10700/12942], Loss: 2.3163, Perplexity: 10.1382
Epoch [1/10], Step [10800/12942], Loss: 1.9977, Perplexity: 7.37238
Epoch [1/10], Step [10900/12942], Loss: 2.3389, Perplexity: 10.3699
Epoch [1/10], Step [11000/12942], Loss: 2.0415, Perplexity: 7.70217
Epoch [1/10], Step [11100/12942], Loss: 2.1721, Perplexity: 8.77664
Epoch [1/10], Step [11200/12942], Loss: 1.9064, Perplexity: 6.72900
Epoch [1/10], Step [11300/12942], Loss: 2.3553, Perplexity: 10.5409
Epoch [1/10], Step [11400/12942], Loss: 3.1134, Perplexity: 22.4963
Epoch [1/10], Step [11500/12942], Loss: 2.1806, Perplexity: 8.85158
Epoch [1/10], Step [11600/12942], Loss: 2.1237, Perplexity: 8.36245
Epoch [1/10], Step [11700/12942], Loss: 2.0054, Perplexity: 7.42940
Epoch [1/10], Step [11800/12942], Loss: 1.9689, Perplexity: 7.16270
Epoch [1/10], Step [11900/12942], Loss: 2.2295, Perplexity: 9.29495
Epoch [1/10], Step [12000/12942], Loss: 2.3271, Perplexity: 10.2486
Epoch [1/10], Step [12100/12942], Loss: 2.3458, 

...

Epoch [2/10], Step [10500/12942], Loss: 2.2344, Perplexity: 9.34059
Epoch [2/10], Step [10600/12942], Loss: 2.0748, Perplexity: 7.96313
Epoch [2/10], Step [10700/12942], Loss: 2.2941, Perplexity: 9.91551
Epoch [2/10], Step [10800/12942], Loss: 1.8470, Perplexity: 6.34088
Epoch [2/10], Step [10900/12942], Loss: 1.9943, Perplexity: 7.34706
Epoch [2/10], Step [11000/12942], Loss: 2.2145, Perplexity: 9.15701
Epoch [2/10], Step [11100/12942], Loss: 2.1477, Perplexity: 8.56484
Epoch [2/10], Step [11200/12942], Loss: 1.8643, Perplexity: 6.45137
Epoch [2/10], Step [11300/12942], Loss: 1.8829, Perplexity: 6.57268
Epoch [2/10], Step [11400/12942], Loss: 1.9331, Perplexity: 6.91121
Epoch [2/10], Step [11500/12942], Loss: 2.1572, Perplexity: 8.64720
Epoch [2/10], Step [11600/12942], Loss: 2.9279, Perplexity: 18.6891
Epoch [2/10], Step [11700/12942], Loss: 2.1641, Perplexity: 8.70666
Epoch [2/10], Step [11800/12942], Loss: 2.1246, Perplexity: 8.36958
Epoch [2/10], Step [11900/12942], Loss: 1.8584, 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch [3/10], Step [2100/12942], Loss: 1.8856, Perplexity: 6.5900
Epoch [3/10], Step [2200/12942], Loss: 1.9139, Perplexity: 6.77976
Epoch [3/10], Step [2300/12942], Loss: 1.9618, Perplexity: 7.11226
Epoch [3/10], Step [2400/12942], Loss: 2.0842, Perplexity: 8.03807
Epoch [3/10], Step [2500/12942], Loss: 1.9554, Perplexity: 7.06653
Epoch [3/10], Step [2600/12942], Loss: 1.8897, Perplexity: 6.61749
Epoch [3/10], Step [2700/12942], Loss: 2.2488, Perplexity: 9.47602
Epoch [3/10], Step [2800/12942], Loss: 1.9894, Perplexity: 7.31109
Epoch [3/10], Step [2900/12942], Loss: 2.0185, Perplexity: 7.52716
Epoch [3/10], Step [3000/12942], Loss: 1.7201, Perplexity: 5.58527
Epoch [3/10], Step [3100/12942], Loss: 1.8311, Perplexity: 6.24080
Epoch [3/10], Step [3200/12942], Loss: 1.9283, Perplexity: 6.87767
Epoch [3/10], Step [3300/12942], Loss: 2.0942, Perplexity: 8.11930
Epoch [3/10], Step [3400/12942], Loss: 2.1195, Perplexity: 8.32719
Epoch [3/10], Step [3500/12942], Loss: 1.7582, Perplexity: 5.80

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch [3/10], Step [6800/12942], Loss: 1.8692, Perplexity: 6.48299
Epoch [3/10], Step [6900/12942], Loss: 1.8927, Perplexity: 6.63756
Epoch [3/10], Step [7000/12942], Loss: 1.9730, Perplexity: 7.19195
Epoch [3/10], Step [7100/12942], Loss: 2.1262, Perplexity: 8.38300
Epoch [3/10], Step [7200/12942], Loss: 1.9388, Perplexity: 6.95033
Epoch [3/10], Step [7300/12942], Loss: 2.1813, Perplexity: 8.85778
Epoch [3/10], Step [7400/12942], Loss: 2.0403, Perplexity: 7.69301
Epoch [3/10], Step [7500/12942], Loss: 1.9691, Perplexity: 7.16415
Epoch [3/10], Step [7600/12942], Loss: 1.8130, Perplexity: 6.12870
Epoch [3/10], Step [7700/12942], Loss: 2.2253, Perplexity: 9.25649
Epoch [3/10], Step [7800/12942], Loss: 1.9846, Perplexity: 7.27612
Epoch [3/10], Step [7900/12942], Loss: 2.2500, Perplexity: 9.48744
Epoch [3/10], Step [8000/12942], Loss: 2.1504, Perplexity: 8.58813
Epoch [3/10], Step [8100/12942], Loss: 2.1501, Perplexity: 8.58566
Epoch [3/10], Step [8199/12942], Loss: 1.9135, Perplexity: 6.7

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.